In [7]:
import h5py
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import pathlib
import matplotlib.pyplot as plt
import numpy as np

In [8]:
batch_size = 32
img_height = 240
img_width = 240

# Change to be directory of training set
directory = r'C:\Users\Brody\Documents\College Material\Fall 2022\BIOE 485\FinalProject\BraTS2020_training_data\content\data\\'
ds_train = tf.data.Dataset.list_files(str(pathlib.Path(directory+'*.h5')))

In [9]:
@tf.function(input_signature=[tf.TensorSpec(None, tf.string)])
def process_path(file_path):
    # need to use non tf functions, call py_function
    image, mask = tf.py_function(func=process_path_helper, inp=[file_path], Tout=[tf.float32, tf.float32])
    image = tf.reshape(image, (240,240,4))
    mask = tf.reshape(mask, (240,240,3))
    return image, mask

def process_path_helper(file_path):
    # arbitrary Python code can be used here
    file_path = file_path.numpy().decode('ascii')
    f = h5py.File(file_path, 'r') 
    image = f['image'][()] 
    mask = f['mask'][()] 
    f.close()
    return image,mask

In [10]:
ds_train = ds_train.map(process_path).batch(batch_size)

In [11]:
def model(inputs):
    
    # model
    l1_in = layers.BatchNormalization()(inputs)
    
    # Level 1
    x1 = layers.Conv2D(filters=32,kernel_size=3,padding='same')(l1_in)
    x1 = keras.activations.relu(x1)
    
    xx1 = layers.Conv2D(filters=32,kernel_size=3,padding='same')(x1)
    xx1 = keras.activations.relu(xx1)
    
    # Level 2
    l2_in = layers.MaxPool2D()(xx1)
    
    x2 = layers.Conv2D(filters=64,kernel_size=3,padding='same')(l2_in)
    x2 = keras.activations.relu(x2)
    
    xx2 = layers.Conv2D(filters=64,kernel_size=3,padding='same')(x2)
    xx2 = keras.activations.relu(xx2)
    
    # Level 3
    l3_in = layers.MaxPool2D()(xx2)
    
    x3 = layers.Conv2D(filters=128,kernel_size=3,padding='same')(l3_in)
    x3 = keras.activations.relu(x3)
    
    xx3 = layers.Conv2D(filters=128,kernel_size=3,padding='same')(x3)
    xx3 = keras.activations.relu(xx3)
    
    # Level 4
    l4_in = layers.MaxPool2D()(xx3)
    
    x4 = layers.Conv2D(filters=256,kernel_size=3,padding='same')(l4_in)
    x4 = keras.activations.relu(x4)
    
    xx4 = layers.Conv2D(filters=256,kernel_size=3,padding='same')(x4)
    xx4 = keras.activations.relu(xx4)
    
    # Level 5
    l5_in = layers.MaxPool2D()(xx4)
    
    x5 = layers.Conv2D(filters=512,kernel_size=3,padding='same')(l5_in)
    x5 = keras.activations.relu(x5)
    
    xx5 = layers.Conv2D(filters=512,kernel_size=3,padding='same')(x5)
    xx5 = keras.activations.relu(xx5)
    
    l5_out = layers.Conv2DTranspose(filters=256,kernel_size=3,padding='same')(xx5)
    l5_out = layers.UpSampling2D()(l5_out)
    
    # Level 4
    y4 = layers.Concatenate()([xx4, l5_out])
    
    yy4 = layers.Conv2D(filters=256,kernel_size=3,padding='same')(y4)
    yy4 = keras.activations.relu(yy4)
    
    yyy4 = layers.Conv2D(filters=256,kernel_size=3,padding='same')(yy4)
    yyy4 = keras.activations.relu(yyy4)
    
    l4_out = layers.Conv2DTranspose(filters=128,kernel_size=3,padding='same')(yyy4)
    l4_out = layers.UpSampling2D()(l4_out)
    
    # Level 3
    y3 = layers.Concatenate()([xx3, l4_out])
    
    yy3 = layers.Conv2D(filters=128,kernel_size=3,padding='same')(y3)
    yy3 = keras.activations.relu(yy3)
    
    yyy3 = layers.Conv2D(filters=128,kernel_size=3,padding='same')(yy3)
    yyy3 = keras.activations.relu(yyy3)
    
    l3_out = layers.Conv2DTranspose(filters=64,kernel_size=3,padding='same')(yyy3)
    l3_out = layers.UpSampling2D()(l3_out)
    
    # Level 2
    y2 = layers.Concatenate()([xx2, l3_out])
    
    yy2 = layers.Conv2D(filters=64,kernel_size=3,padding='same')(y2)
    yy2 = keras.activations.relu(yy2)
    
    yyy2 = layers.Conv2D(filters=64,kernel_size=3,padding='same')(yy2)
    yyy2 = keras.activations.relu(yyy2)
    
    l2_out = layers.Conv2DTranspose(filters=32,kernel_size=3,padding='same')(yyy2)
    l2_out = layers.UpSampling2D()(l2_out)
    
    # Level 1
    y1 = layers.Concatenate()([xx1, l2_out])
    
    yy1 = layers.Conv2D(filters=32,kernel_size=3,padding='same')(y1)
    yy1 = keras.activations.relu(yy1)
    
    yyy1 = layers.Conv2D(filters=32,kernel_size=3,padding='same')(yy1)
    yyy1 = keras.activations.relu(yyy1)
    
    outputs = layers.Conv2D(filters=3,kernel_size=1,padding='same')(yyy1)
    
    model = keras.Model(inputs=inputs, outputs = outputs)
    return model

In [12]:
inputs = keras.Input(shape=(240,240,4))
model = model(inputs)

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"]
)
model.summary()
model.fit(ds_train, epochs=1)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 240, 240, 4  0           []                               
                                )]                                                                
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 240, 240, 4)  16         ['input_2[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 conv2d_2 (Conv2D)              (None, 240, 240, 32  1184        ['batch_normalization_2[0][0]']  
                                )                                                           

 spose)                                                                                           
                                                                                                  
 up_sampling2d_1 (UpSampling2D)  (None, 60, 60, 128)  0          ['conv2d_transpose_1[0][0]']     
                                                                                                  
 concatenate_1 (Concatenate)    (None, 60, 60, 256)  0           ['tf.nn.relu_7[0][0]',           
                                                                  'up_sampling2d_1[0][0]']        
                                                                                                  
 conv2d_14 (Conv2D)             (None, 60, 60, 128)  295040      ['concatenate_1[0][0]']          
                                                                                                  
 tf.nn.relu_14 (TFOpLambda)     (None, 60, 60, 128)  0           ['conv2d_14[0][0]']              
          

In [13]:
# Change to filepath of model to save
model.save(r'C:\Users\Brody\Documents\College Material\Fall 2022\BIOE 485\FinalProject\models\unet_model.h5')

   7/1410 [..............................] - ETA: 8:38:56 - loss: 0.4527 - accuracy: 0.3924

KeyboardInterrupt: 

In [ ]:
# Change to directory of of test set
directory = r'C:\Users\Brody\Documents\College Material\Fall 2022\BIOE 485\FinalProject\BraTS2020_training_data\content\test\\'
ds_test = tf.data.Dataset.list_files(str(pathlib.Path(directory+'*.h5')))
batch_size = 1
ds_test = ds_test.map(process_path).batch(batch_size)
model.evaluate(ds_test)